# Performing Variant Calling

When performing variant calling we need the aligned sequences in SAM, BAM or CRAM format and the reference genome that we want to call varants against.

First, check you are in the correct directory. 

In [ ]:
pwd

It should display something like:

  ``/home/manager/course_data/variant_calling/data``

## Assessing the input data

 To list the files in the current directory, type

In [ ]:
ls -lh

The listing shows aligned data for two mouse strains A/J and NZO (A_J.bam and NZO.bam) and the chromosome 19 of the mouse reference genome (GRCm38_68.19.fa).

Before performing variant calling, it is important to check the quality of the data that you will be working with. We have already seen how to do this in the QC and Data Formats and Read Alignment sessions. The commands would look like:

`samtools stats -r GRCm38_68.19.fa A_J.bam > A_J.stats`

`samtools stats -r GRCm38_68.19.fa NZO.bam > NZO.stats`

`plot-bamstats -r GRCm38_68.19.fa.gc -p A_J.graphs/ A_J.stats`

`plot-bamstats -r GRCm38_68.19.fa.gc -p NZO.graphs/ NZO.stats`

You do not need to run these QC checks on this data and for this we will assume that QC has already been performed and the data is of good quality.

## Generating pileup

The command `samtools mpileup` prints the read bases that align to each position in the reference genome. Type the command:


In [ ]:
samtools mpileup -f GRCm38_68.19.fa A_J.bam | less -S

Each line corresponds to a position on the genome. 

The columns are: chromosome, position, reference base, read depth, read bases (dot . and comma , indicate match on the forward and on the reverse strand; ACGTN and acgtn a mismatch on the forward and the reverse strand) and the final column is the base qualities encoded into characters. The caret symbol ^ marks the start of a read, the dollar sign $ the end of a read, deleted bases are represented by asterisk *.

This output can be used for a simple consensus calling. One rarely needs this type of output. Instead, for a more sophisticated variant calling method, see the next section.

### Exercises
Look at the output from the `mpileup` command above and answer the following questions.

**Q1:** What is the read depth at position 10001994? (Rather than scrolling to the position, use the substring searching capabilities of less: press /, then type 10001994 followed by enter to find the position.)

**Q2:** What is the reference allele and the alternate allele at position 10001994? 

**Q3:** How many reads call the reference allele at position 10001994 and how many reads call the alternate allele at position 10001994?

## Generating genotype likelihoods and calling variants

The `bcftools mpileup` command can be used to generate genotype likelihoods. (Beware: the command mpileup is present in both samtools and bcftools, but in both they do different things. While `samtools mpileup` produces the text pileup output seen in the previous exercise, `bcftools mpileup` generates a VCF file with genotype likelihoods.)

Run the following command (when done, press q to quit the viewing mode):

In [ ]:
bcftools mpileup -f GRCm38_68.19.fa A_J.bam | less -S

This generates an intermediate output which contains genotype likelihoods and other raw information necessary for variant calling. This output is usually streamed directly to the caller like this

 `bcftools mpileup -f GRCm38_68.19.fa A_J.bam | bcftools call -m | less -S`

The output above contains both variant and non-variant positions. Check the input/output options section of the `bcftools call` usage page and see if there is an option to print out only variant sites. Then construct a command to print out variant sites only:

The INFO and FORMAT fields of each entry tells us something about the data at the position in the genome. It consists of a set of key-value pairs with the tags being explained in the header of the VCF file (see the ##INFO and ##FORMAT lines in the header).

We can tell mpileup to add additional ##INFO and ##FORMAT information to the output. For example, we can ask it to add the FORMAT/AD tag which informs about the number of high-quality reads that support alleles listed in REF and ALT columns. The list of all available tags can be printed with the command:

In [ ]:
bcftools mpileup -a ?

Now let's run the variant calling again, this time adding the `-a AD` option. We will also add the `-Ou` option so that it streams a binary uncompressed BCF into call. This is to avoid the unnecessary CPU overhead of formatting the internal binary format to plain text VCF only to be immediately formatted back to the internal binary format again.

In [ ]:
bcftools mpileup -a AD -f GRCm38_68.19.fa A_J.bam -Ou | bcftools call -mv -o out.vcf

### Exercises

Look at the content of the VCF file produced above and answers the questions that follow. 

In [ ]:
less -S out.vcf

**Q1:** What is the reference allele and the alternate allele at position 10001994?

**Q2:** What is the total raw read depth at position 10001994? 

**Note:** This number may be different from the values we obtained earlier, because some low quality reads or bases might have been filtered previously.

**Q3:** What is the number of high-quality reads supporting the SNP call at position 10001994? How many reads support the reference allele and how many support the alternate allele? 

**Hint:** Look up the AD tag in the FORMAT column: the first value gives the number of reads calling the reference allelle and the second gives the number of reads calling the alternate alleles.

**Q4:** What sort of event is happening at position 10003649? 

Congratulations, you have sucessfully called variants from some NGS data. Now continue to the next section of the tutorial: [filtering variants](filtering.ipynb)